In [1]:
!pip install datasets
from datasets import Dataset, load_dataset
ds = load_dataset("cnamuangtoun/resume-job-description-fit") # ["train", "test"]["resume_text", "job_description_text", "label"]
print(ds)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


train.csv:   0%|          | 0.00/53.4M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/15.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6241 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1759 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['resume_text', 'job_description_text', 'label'],
        num_rows: 6241
    })
    test: Dataset({
        features: ['resume_text', 'job_description_text', 'label'],
        num_rows: 1759
    })
})


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B", device_map="auto")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B", device_map="auto")

input_text = "Once upon a time"

inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=20)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Once upon a time, a man in a cave in the Himalayas, in a land called


In [5]:
def prepare_dataset(examples):
    """
    Combines resume_text, job_description_text, and label into a prompt.
    """
    prompt_template = "Given the resume:\n\n{resume_text}\n\nAnd the job description:\n\n{job_description}\n\nTell if the resume is a good fit:\n\n"
    examples["prompt"] = [
        prompt_template.format(
            resume_text=resume_text,
            job_description=job_description,
        )
        for resume_text, job_description in zip(
            examples["resume_text"], examples["job_description_text"]
        )
    ]
    examples["labels"] = examples["label"]
    return examples

def tokenize_function(examples):
    tokenizer.pad_token = tokenizer.eos_token
    return tokenizer(examples["prompt"], padding=True, truncation=True, return_tensors="pt", return_attention_mask=True)

train_subset_indices = list(range(0, len(ds["train"]), 100))
train_subset_ds = ds["train"].select(train_subset_indices)

train_subset_ds = train_subset_ds.map(prepare_dataset, batched=True)
tokenized_train_subset = train_subset_ds.map(tokenize_function, batched=True)
tokenized_train_subset = tokenized_train_subset.remove_columns(["resume_text", "job_description_text", "label", "prompt"])

test_subset_indices = list(range(0, len(ds["test"]), 100))
test_subset_ds = ds["test"].select(test_subset_indices)

test_subset_ds = test_subset_ds.map(prepare_dataset, batched=True)
tokenized_test_subset = test_subset_ds.map(tokenize_function, batched=True)
tokenized_test_subset = tokenized_test_subset.remove_columns(["resume_text", "job_description_text", "label", "prompt"])

print(tokenized_train_subset)
print(tokenized_test_subset)

Map:   0%|          | 0/63 [00:00<?, ? examples/s]

Map:   0%|          | 0/63 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 63
})
Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 18
})


In [6]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_subset,
    eval_dataset=tokenized_test_subset
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


OutOfMemoryError: CUDA out of memory. Tried to allocate 258.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 245.06 MiB is free. Process 4220 has 14.51 GiB memory in use. Of the allocated memory 14.20 GiB is allocated by PyTorch, and 189.33 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)